In [5]:
import pandas as pd
import numpy as np


In [8]:
# для всех норм по ТПГ нужно объявить эту переменную
row_number=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,usecols=[5])
skip_row=row_number.ЦОК.tolist()
skip_row_n=skip_row.index('подгруппа')-2

In [7]:
#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ (всех)
df=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)
df_all=df[["статус","Номер из NAV","Адрес магазина","дата планового открытия"]]
df_all=df_all[~df_all.статус.isin(['Закрыт','закрыт'])]
df_all=df_all[['статус','Номер из NAV']].groupby(by='статус').count().reset_index(drop=True, inplace=False)

#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ иф/бк(золото)
a=[2,3,6,18]+[*range(28,41)]
a_shops=df.iloc[1:, lambda df: a]
del a_shops['ИФ аксессуары']
a_shops=a_shops[a_shops.статус.isin(['открыт','Отгружено','склад'])]
a_shops['сумма']=a_shops.iloc[:,3:].sum(axis=1)
a_shops=a_shops[a_shops['сумма'] >0]
a_shops_count=a_shops[['статус','Номер из NAV']].groupby(by=['статус']).count().reset_index(drop=True, inplace=False)

#Переворачиваем и соединяем золото и серебро
final = df_all.rename(columns={'Номер из NAV':'с_серебром'}).T.append(a_shops_count.rename(columns={'Номер из NAV':'золото'}).T)
final.at['с_серебром',1] = final.at['с_серебром',1]-3 # удаляем 3 ломбарда
final[3]=final[0]+final[2]
final[4]=final[1]+final[3]
final = final.drop([0,2],axis=1).rename(columns={1:'открыто',3:'к_открытию',4:'итого'})

#ДОСТАЕТ СПИСОК НОВЫХ МАГОВ
new_shops=a_shops[["статус","Номер из NAV","Адрес магазина","дата планового открытия"]]
new_shops=new_shops[
    (new_shops.статус == 'Отгружено')|(new_shops.статус == 'склад')].drop(columns='статус').sort_values('дата планового открытия')

#ДОСТАЕТ НОРМЫ ПО магам иф/бк

norm_all=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)
list=norm_all.columns.tolist()

cols=[list[list.index('Название из NAV')]]+list[list.index(1001):list.index(1102)+1]+list[list.index(1501):list.index(1593)+1]

opened=a_shops[['Номер из NAV','статус']]
norm_all=norm_all[cols]
merged=pd.merge(opened,norm_all,left_on='Номер из NAV', right_on='Название из NAV',how = 'left')
del merged['Название из NAV'],merged['Номер из NAV']
merged=merged.groupby(by='статус').sum()
merged=merged.T
merged=merged.assign(sum_all=merged['Отгружено']+merged['открыт']+merged['склад'])
del merged['Отгружено'],merged['склад']

# СОХРАНЯЕМ ВСЕ В 1 ФАЙЛ

with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\для_новые_нормы.xlsx'
) as writer:
    final.to_excel(writer, sheet_name='количество')
    new_shops.to_excel(writer, sheet_name='новые_маги')
    merged.to_excel(writer, sheet_name='новые_нормы')


In [3]:
#ДОСТАЕТ НОРМЫ ПО БУКВАМ
norm=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)
list=norm.columns.tolist()
cols=[list[list.index('Название из NAV')]]+list[list.index(1088):list.index(1088)+3]+list[list.index(1582):list.index(1582)+3]
norm=norm[cols]
norm=norm.assign(sum_if=norm[1088]+norm[1089]+norm[1090],sum_bk=norm[1582]+norm[1583]+norm[1584])
norm.set_index('Название из NAV',drop=True, inplace=True)
norm.to_excel(r'C:\Остатки\norm_b.xlsx')

In [4]:
#ДОСТАЕТ НОРМЫ ПО ЗОДИАКАМ
normb=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)
listb=normb.columns.tolist()
colsb=[listb[listb.index('Название из NAV')]]+listb[listb.index(1091):listb.index(1091)+3]+listb[listb.index(1585):listb.index(1585)+3]
normb=normb[colsb]
normb=normb.assign(sum_if1=normb[1091]+normb[1092]+normb[1093],sum_bk1=normb[1585]+normb[1586]+normb[1587])
normb.set_index('Название из NAV',drop=True, inplace=True)
normb=normb[[1091,1092,1093,'sum_if1',1585,1586,1587,'sum_bk1']]
normb.to_excel(r'C:\Остатки\norm_z.xlsx')


In [10]:
#ВЫДАЕТ СПИСОК АКТУАЛЬНЫХ МАГОВ ИФ\БК С НОРМАМИ ПО ТГ И ДАТЫ ОТКРЫТИЯ

#выгружаем датафрэйм с актуальными нормами ИФ БК из реестра магов 
all_shops = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=[2,3,18]+[*range(28,34)]+[*range(35,41)])

#выбираем открытые маги с суммой норм по иф/бк больше нуля
all_shops = all_shops[all_shops.статус.isin(['открыт','Отгружено','склад'])]
all_shops['сумма'] = all_shops.iloc[:,3:].sum(axis=1)
all_shops = all_shops[all_shops['сумма'] >0]
del all_shops['статус'], all_shops['сумма']
all_shops = all_shops.rename(columns={'Номер из NAV':'Код'})
all_shops['Код'] = all_shops['Код'].astype(np.int64)

# делим на два отдельных иф и бк
cols = all_shops.columns.tolist()
shops_if = all_shops[cols[:5]+[cols[7],cols[6],cols[5]]]
shops_bk = all_shops[cols[:2]+cols[8:11]+[cols[13],cols[12],cols[11]]]

#выгружаем из файла "наполняемость" актуальную структуру магов иф/бк, добавляет туда нормы и новые маги из датафрэйма выше
shops_if_list = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='ИФ',skiprows=3, usecols=[2],keep_default_na=False)

shops_bk_list = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='БК',skiprows=3, usecols=[2],keep_default_na=False)

# список новых магов которые надо добавить
add_to_if = pd.DataFrame({'Код':[x for x in shops_if['Код'].tolist() if x not in shops_if_list['Код'].tolist()]})
add_to_bk = pd.DataFrame({'Код':[x for x in shops_bk['Код'].tolist() if x not in shops_bk_list['Код'].tolist()]})

# добавляем новые маги в структуру
shops_if_list = shops_if_list.append(add_to_if, sort=False)
shops_bk_list = shops_bk_list.append(add_to_bk, sort=False)

#соединяем "структуру" и нормы
shops_if = pd.merge(shops_if_list,shops_if,how='left',on='Код')
shops_bk = pd.merge(shops_bk_list,shops_bk,how='left',on='Код')


#удаляем индекс и сохраняем на отдельные вкладки
shops_if.set_index('Код', drop=True,inplace=True)
shops_bk.set_index('Код', drop=True,inplace=True)

#создаем датафрейм из магов с датами открытия последние 90 дней
from datetime import date, timedelta

three_month_ago = date.today() - timedelta(90)
dates=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols = [2,6,18])
dates.rename(columns={'дата планового открытия':'дата'}, inplace=True)

dates=dates[(
    dates.статус == 'Отгружено')|(
    dates.статус == 'склад')|(
    dates.статус == 'открыт')&(
    dates.дата > pd.Timestamp(three_month_ago
                             ))]
del dates['статус']
dates['дата'] = dates.дата.apply(lambda x:x.date())
dates.set_index('Номер из NAV', drop=True,inplace=True)

#сохраняем все в эксель
with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\актуальные маги.xlsx'
) as writer:
    shops_if.to_excel(writer, sheet_name='ИФ')
    shops_bk.to_excel(writer, sheet_name='БК')
    dates.to_excel(writer, sheet_name='даты')

In [47]:
# СПИСОК НОРМ ПО ТГ

#читает файл с реестром
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)

#вынимает нужные две строки и сохраняет
rep_tn = df.iloc[0:1,28:73]
rep_tn.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\Рейтинг ТГ и ТН\123.xlsx')

In [5]:
!pip install emoji

from emoji import emojize

print(emojize(":thumbs_up:"))

👍


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
xlwings 0.15.2 has requirement pywin32>=224, but you'll have pywin32 223 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [160]:
mus=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\мусульманкэ.xlsx',
    sheet_name='Лист1')
mus



,Название из NAV,1055,1056,1057,1058,1059,1060,1061,1062,1063,...,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572
0,789.0,2.038015,6.793384,4.755369,15.205548,11.310957,5.21276,52.754625,45.873587,16.055755,...,2.009433,62.973673,44.081571,18.892102,12.846861,14.988005,14.988005,13.854208,12.314852,4.618069
1,3081.0,1.972311,6.574369,4.602059,7.602774,5.655479,2.60638,51.053849,44.394651,15.538128,...,2.326864,72.921659,51.045161,21.876498,14.876287,17.355668,17.355668,16.042765,14.260235,5.347588
2,3272.0,2.065490,6.884968,4.819478,0.000000,0.000000,0.00000,53.465830,46.492026,16.272209,...,2.561425,80.272590,56.190813,24.081777,16.375904,19.105221,19.105221,17.659970,15.697751,5.886657
3,3653.0,1.621691,5.405636,3.783945,3.801387,2.827739,1.30319,41.977949,36.502564,12.775898,...,1.476076,46.258807,32.381165,13.877642,9.436967,11.009795,11.009795,10.176938,9.046167,3.392313
4,3954.0,1.336775,4.455918,3.119142,7.602774,5.655479,2.60638,34.602825,30.089413,10.531294,...,1.144088,35.854610,25.098227,10.756383,7.314473,8.533551,8.533551,7.888014,7.011568,2.629338
5,4345.0,2.542142,8.473807,5.931665,0.000000,0.000000,0.00000,65.804098,57.220955,20.027334,...,1.582057,49.580129,34.706090,14.874039,10.114529,11.800284,11.800284,10.907628,9.695670,3.635876
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,1057
3,3.783945


Название из NAV    3081.000000
1055                  1.972311
1056                  6.574369
1057                  4.602059
1058                  7.602774
1059                  5.655479
1060                  2.606380
1061                 51.053849
1062                 44.394651
1063                 15.538128
1558                  2.647989
1559                  5.146186
1560                  0.341193
1561                 39.002040
1562                 36.454199
1563                  2.326864
1564                 72.921659
1565                 51.045161
1566                 21.876498
1567                 14.876287
1568                 17.355668
1569                 17.355668
1570                 16.042765
1571                 14.260235
1572                  5.347588
Name: 1, dtype: float64

In [49]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Yandex")
location = geolocator.geocode("Владикавказ")
print(location.address)

Владикавказ, городской округ Владикавказ, Северная Осетия — Алания, СКФО, РФ


In [159]:
list=list(range(1,10,1)
df=pd.DataFrame({'col':list})



SyntaxError: invalid syntax (<ipython-input-159-9976a0f31040>, line 2)

In [4]:
import antigravity